# Verify the functionality of encoding Functions

In [ ]:
from acousticNN.model.encoding import pos_enc_sincos, SinosoidalEncoding, PositionalEncoding, FourierEncoding, RandomEncoding
import torch
import numpy as np
import matplotlib.pyplot as plt

## test input output

In [ ]:
b, num_tokens = np.random.randint(40, 122, 2)
size = (b, num_tokens)
dim = np.random.choice(np.arange(50, 256, 2))
base = np.random.uniform(0, 10000)
random_tensor = torch.from_numpy(np.random.uniform(-1, 5, size)).float()
random_tensor = torch.sort(random_tensor, dim=1).values
random_tensor_idx = torch.arange(num_tokens).unsqueeze(0).repeat(b, 1).int()
print(b, num_tokens, dim, base)

In [ ]:

pos_encoder = PositionalEncoding(dim=dim, base=base)
sin_encoder = SinosoidalEncoding(dim=dim, base=base)
fourier_encoder = FourierEncoding(dim=dim, n_parameters=num_tokens, factor_pars=(0, 1))
random_encoder = RandomEncoding(dim=dim, factor_pars=(0, 1))

In [ ]:
assert torch.all(pos_encoder(random_tensor) == sin_encoder(random_tensor_idx))
assert pos_encoder(random_tensor).shape == (b, num_tokens, dim)
assert sin_encoder(random_tensor).shape == (b, num_tokens, dim)
assert fourier_encoder(random_tensor).shape == (b, num_tokens, dim)
assert random_encoder(random_tensor).shape == (b, num_tokens, dim)

## test base value

In [ ]:
pos_encoder_base = 10000
sin_encoder_base = 0.01
pos_encoder = PositionalEncoding(dim=dim, base=pos_encoder_base)
sin_encoder = SinosoidalEncoding(dim=dim, base=sin_encoder_base)
fourier_encoder = FourierEncoding(dim=dim, n_parameters=num_tokens, factor_pars=(0, 1))
random_encoder = RandomEncoding(dim=dim, factor_pars=(0, 1))

In [ ]:
for encoder in [pos_encoder, sin_encoder, fourier_encoder, random_encoder]:
    encoding = encoder(random_tensor)
    med = torch.median(torch.std(encoding.view(b*num_tokens, -1), dim=0))
    print(med)
    plt.imshow(encoder(random_tensor)[0], cmap="coolwarm")
    plt.xlabel('dim')
    plt.ylabel('tokens')
    plt.colorbar()
    plt.show()